In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import matplotlib.pyplot as plt
from PIL import Image
from datetime import *
from datetime import datetime, timedelta, date
%load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [4]:
pip install fastf1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.4/130.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 36.4 MB/s eta 0:00:00


In [5]:
import fastf1
session = fastf1.get_session(2021, 'Spanish Grand Prix', 'Q')
session.load()

circuit_info = session.get_circuit_info()

req         WARNING 	DEFAULT CACHE ENABLED! (24.0 KB) /root/.cache/fastf1
core           INFO 	Loading data for Spanish Grand Prix - Qualifying [v3.3.8]
INFO:fastf1.fastf1.core:Loading data for Spanish Grand Prix - Qualifying [v3.3.8]
req            INFO 	No cached data found for session_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
INFO:fastf1.api:Fetching session info data...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
INFO:fastf1.api:Fetching driver list...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for session_status_

In [6]:
query2=f"""
SELECT
A.Year,
A.GP,
A.Driver,
A.DriverNumber,
A.Team,
CAST(A.Position AS INT64) Position,
Status,
A.LapStartTime,
A.Time LapEndTime,
B.Time as LapInstant,
A.LapTime,
CAST(LapNumber as INT64) LapNumber,
RPM,
Speed,
nGear,
Throttle,
Brake,
DRS,
Distance,
RelativeDistance,
DriverAhead,
DistanceToDriverAhead,
X,
Y,
Z,
CAST(Stint as INT64) Stint,
PitOutTime,
PitInTime,
Sector1Time,
Sector2Time,
Sector3Time,
Sector1SessionTime,
Sector2SessionTime,
Sector3SessionTime,
CAST(SpeedI1 AS INT64) SpeedI1,
CAST(SpeedI2 AS INT64) SpeedI2,
CAST(SpeedFL AS INT64) SpeedFL,
CAST(SpeedST AS INT64) SpeedST,
Compound,
CAST(TyreLife AS INT64) TyreLife,
FreshTyre,
TrackStatus,
Deleted,
DeletedReason
FROM `perceptive-ivy-290216.f1_api.qualifying_lap_time`  A
INNER JOIN
`perceptive-ivy-290216.f1_api.qualifying_telemetry_2021` B
ON B.SessionTime>=A.LapStartTime AND B.SessionTime<=A.Time
AND A.Driver=B.Driver
AND A.Year=B.Year
AND A.GP=B.GP
WHERE A.Year=2021
AND A.GP="Spanish Grand Prix"
# AND A.DRIVER='HAM'
AND B.Year=2021
AND B.GP="Spanish Grand Prix"
# AND B.Driver='HAM'
ORDER BY LapNumber, LapStartTime, LapInstant
"""
track2=pd.read_gbq(query2,project_id,dialect='standard')

In [7]:
#Convert braking from True False to 0 & 1
def braking_ind(ind):
  if ind==True:
    return 1
  else:
    return 0

track2['Braking'] = track2['Brake'].apply(braking_ind)

In [8]:
# track2["Time"]=track2['Time'].str.split('days ').str[1]
# track2["Time"]=pd.to_datetime(track2['Time'],format='%H:%M:%S.%f')
# track2["Time"]=track2["Time"].dt.strftime("%H:%M:%S.%f")

track2["LapTime"]=track2['LapTime'].str.split('days ').str[1]
track2["LapTime"]=pd.to_datetime(track2['LapTime'],format='%H:%M:%S.%f')
track2["LapTime"]=track2["LapTime"].dt.strftime("%H:%M:%S.%f")

# track2["Sector1Time"]=track2['Sector1Time'].str.split('days ').str[1]
# track2["Sector1Time"]=pd.to_datetime(track2['Sector1Time'],format='%H:%M:%S.%f')
# track2["Sector1Time"]=track2["Sector1Time"].dt.strftime("%H:%M:%S.%f")

# track2["Sector2Time"]=track2['Sector2Time'].str.split('days ').str[1]
# track2["Sector2Time"]=pd.to_datetime(track2['Sector2Time'],format='%H:%M:%S.%f')
# track2["Sector2Time"]=track2["Sector2Time"].dt.strftime("%H:%M:%S.%f")

# track2["Sector3Time"]=track2['Sector3Time'].str.split('days ').str[1]
# track2["Sector3Time"]=pd.to_datetime(track2['Sector3Time'],format='%H:%M:%S.%f')
# track2["Sector3Time"]=track2["Sector3Time"].dt.strftime("%H:%M:%S.%f")

# track2["Sector1SessionTime"]=track2['Sector1SessionTime'].str.split('days ').str[1]
# track2["Sector1SessionTime"]=pd.to_datetime(track2['Sector1SessionTime'],format='%H:%M:%S.%f')
# track2["Sector1SessionTime"]=track2["Sector1SessionTime"].dt.strftime("%H:%M:%S.%f")

# track2["Sector2SessionTime"]=track2['Sector2SessionTime'].str.split('days ').str[1]
# track2["Sector2SessionTime"]=pd.to_datetime(track2['Sector2SessionTime'],format='%H:%M:%S.%f')
# track2["Sector2SessionTime"]=track2["Sector2SessionTime"].dt.strftime("%H:%M:%S.%f")

# track2["Sector3SessionTime"]=track2['Sector3SessionTime'].str.split('days ').str[1]
# track2["Sector3SessionTime"]=pd.to_datetime(track2['Sector3SessionTime'],format='%H:%M:%S.%f')
# track2["Sector3SessionTime"]=track2["Sector3SessionTime"].dt.strftime("%H:%M:%S.%f")

# track2["LapStartTime"]=track2['LapStartTime'].str.split('days ').str[1]
# track2["LapStartTime"]=pd.to_datetime(track2['LapStartTime'],format='%H:%M:%S.%f')
# track2["LapStartTime"]=track2["LapStartTime"].dt.strftime("%H:%M:%S.%f")

# track2["LapEndTime"]=track2['LapEndTime'].str.split('days ').str[1]
# track2["LapEndTime"]=pd.to_datetime(track2['LapEndTime'],format='%H:%M:%S.%f')
# track2["LapEndTime"]=track2["LapEndTime"].dt.strftime("%H:%M:%S.%f")

# track2["LapInstant"]=track2['LapInstant'].str.split('days ').str[1]
# track2["LapInstant"]=pd.to_datetime(track2['LapInstant'],format='%H:%M:%S.%f')
# track2["LapInstant"]=track2["LapInstant"].dt.strftime("%H:%M:%S.%f")

# track2["PitOutTime"]=track2['PitOutTime'].str.split('days ').str[1]
# track2["PitOutTime"]=pd.to_datetime(track2['PitOutTime'],format='%H:%M:%S.%f')
# track2["PitOutTime"]=track2["PitOutTime"].dt.strftime("%H:%M:%S.%f")

# track2["PitInTime"]=track2['PitInTime'].str.split('days ').str[1]
# track2["PitInTime"]=pd.to_datetime(track2['PitInTime'],format='%H:%M:%S.%f')
# track2["PitInTime"]=track2["PitInTime"].dt.strftime("%H:%M:%S.%f")

# race_lap_time["update"]=race_lap_time["update"].dt.strftime("%H:%M:%S.%f")
# race_lap_time["update2"]=race_lap_time["update2"].dt.strftime("%H:%M:%S.%f")

# race_lap_time["update"]=pd.to_time(race_lap_time['update'],format='%H:%M:%S.%f')

In [9]:
#Convert lapnumber to string for discrete chart coloring
track2["LapNumber"]=track2["LapNumber"].astype(str)

In [10]:
track2['Dist_Diff2'] = track2.groupby(["LapNumber","Driver"])["Distance"].shift(1)
# track2['Dist_Diff2'] = track2.groupby(["Driver"])["Distance"].shift(1) ## Use when not looking at just 1 lap
track2['Dist_Diff'] = track2['Distance']-track2["Dist_Diff2"]
track2["Dist_Diff_Sum"] = track2.groupby(by=["LapNumber","Driver"])["Dist_Diff"].cumsum()
# track2["Dist_Diff_Sum"] = track2.groupby(by=["Driver"])["Dist_Diff"].cumsum()  ## Use when not looking at just 1 lap

##Another way to calculate distances manually, if the distance column wasnt present in the data set. Dist_Diff_Sum AND Dist_Total Should match very closely which they do, even when using our own method

#Calculate the distance traversed in totality and then for each lap
track2['X_Diff'] = track2['X'] - track2['X'].shift(1)
track2['Y_Diff'] = track2['Y'] - track2['Y'].shift(1)
track2['Z_Diff'] = track2['Z'] - track2['Z'].shift(1)
track2['X_Diff']=track2['X_Diff'].pow(2)
track2['Y_Diff']=track2['Y_Diff'].pow(2)
track2['Z_Diff']=track2['Z_Diff'].pow(2)
track2['Dist']=track2['X_Diff']+track2['Y_Diff']+track2['Z_Diff']

#Distance between current point and previous data point
track2['Dist']=track2['Dist'].pow(1./2)
track2['Dist']=track2['Dist']/10

#Running sum of distance covered by a driver
track2['Dist_Total']=track2['Dist'].cumsum()

#Running sum of distance covered by a driver by lap
track2["Dist_Lap"] = track2.groupby(by=["LapNumber","Driver"])["Dist"].cumsum()

# #Drop columns that are not necessary anymore
# track2.drop(['X_Diff','Y_Diff','Z_Diff','Dist_Diff2'], axis=1, inplace=True)

# #Assign Rank for each entry point
track2["RK"] = track2.groupby(by=["LapNumber","Driver"])["LapInstant"].rank(method="dense", ascending=True)
track2["RK"]= track2["RK"].astype(int)
track2=track2.sort_values(by=['Driver','LapInstant'])

In [11]:
# track2[(track2["Driver"]=='HAM')&((track2["LapNumber"]=='1')|(track2["LapNumber"]=='2'))].sort_values(by=["LapNumber","LapInstant"])

In [12]:
track2[track2["Driver"]=='HAM'].head()

,Year,GP,Driver,DriverNumber,Team,Position,Status,LapStartTime,LapEndTime,LapInstant,LapTime,LapNumber,RPM,Speed,nGear,Throttle,Brake,DRS,Distance,RelativeDistance,DriverAhead,DistanceToDriverAhead,X,Y,Z,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,Compound,TyreLife,FreshTyre,TrackStatus,Deleted,DeletedReason,Braking,Dist_Diff2,Dist_Diff,Dist_Diff_Sum,X_Diff,Y_Diff,Z_Diff,Dist,Dist_Total,Dist_Lap,RK
6047,2021,Spanish Grand Prix,HAM,44,Mercedes,<NA>,OnTrack,0 days 00:28:26.021000,0 days 00:30:03.977000,0 days 00:00:00,NaN,1,6827,79,2,45,False,8,-0.008707,-9.875533e-08,,403.668333,-867,-2822,1878,1,0 days 00:28:26.021000,NaT,NaT,0 days 00:00:33.296000,0 days 00:00:37.575000,NaT,0 days 00:29:26.402000,0 days 00:30:04.234000,225,255,283,158,MEDIUM,1,True,1,False,,0,NaN,NaN,NaN,3125824,6100900,29241,304.236175,28701.050126,304.236175,1
6048,2021,Spanish Grand Prix,HAM,44,Mercedes,<NA>,OnTrack,0 days 00:28:26.021000,0 days 00:30:03.977000,0 days 00:00:00.013000,NaN,1,6963,79,2,44,False,8,0.279605,3.171160e-06,,403.668333,-869,-2825,1878,1,0 days 00:28:26.021000,NaT,NaT,0 days 00:00:33.296000,0 days 00:00:37.575000,NaT,0 days 00:29:26.402000,0 days 00:30:04.234000,225,255,283,158,MEDIUM,1,True,1,False,,0,-0.008707,0.288312,0.288312,4,9,0,0.360555,28701.410681,304.59673,2
6049,2021,Spanish Grand Prix,HAM,44,Mercedes,<NA>,OnTrack,0 days 00:28:26.021000,0 days 00:30:03.977000,0 days 00:00:00.135000,NaN,1,7235,80,2,42,False,8,3.029167,3.435552e-05,,403.668333,-883,-2847,1877,1,0 days 00:28:26.021000,NaT,NaT,0 days 00:00:33.296000,0 days 00:00:37.575000,NaT,0 days 00:29:26.402000,0 days 00:30:04.234000,225,255,283,158,MEDIUM,1,True,1,False,,0,0.279605,2.749562,3.037874,196,484,1,2.609598,28704.020279,307.206328,3
6050,2021,Spanish Grand Prix,HAM,44,Mercedes,<NA>,OnTrack,0 days 00:28:26.021000,0 days 00:30:03.977000,0 days 00:00:00.233000,NaN,1,7424,82,2,40,False,8,5.295426,6.005847e-05,,403.668333,-896,-2866,1878,1,0 days 00:28:26.021000,NaT,NaT,0 days 00:00:33.296000,0 days 00:00:37.575000,NaT,0 days 00:29:26.402000,0 days 00:30:04.234000,225,255,283,158,MEDIUM,1,True,1,False,,0,3.029167,2.266259,5.304133,169,361,1,2.304344,28706.324622,309.510671,4
6051,2021,Spanish Grand Prix,HAM,44,Mercedes,<NA>,OnTrack,0 days 00:28:26.021000,0 days 00:30:03.977000,0 days 00:00:00.375000,NaN,1,7613,85,2,39,False,8,8.695833,9.862445e-05,9,403.668333,-913,-2894,1878,1,0 days 00:28:26.021000,NaT,NaT,0 days 00:00:33.296000,0 days 00:00:37.575000,NaT,0 days 00:29:26.402000,0 days 00:30:04.234000,225,255,283,158,MEDIUM,1,True,1,False,,0,5.295426,3.400408,8.704541,289,784,0,3.275668,28709.60029,312.786339,5


In [13]:
# pd.DataFrame(track2.groupby(by=["Driver"])["LapTime"].unique())
Lap_Driver=pd.DataFrame(track2.groupby(by=["Driver","LapNumber"])["LapTime"].unique().explode())
Lap_Driver=Lap_Driver.sort_values(by=["Driver","LapTime"])
Lap_Driver["RK"] = Lap_Driver.groupby("Driver")["LapTime"].rank(method="dense", ascending=True)
Lap_Driver.sort_values(by=["LapTime"])

LapTime    RK
Driver LapNumber                       
HAM    15         00:01:16.741000   1.0
VER    10         00:01:16.777000   1.0
BOT    12         00:01:16.873000   1.0
VER    7          00:01:16.922000   2.0
       13         00:01:16.938000   3.0
HAM    18         00:01:17.039000   2.0
BOT    15         00:01:17.134000   2.0
       9          00:01:17.142000   3.0
HAM    12         00:01:17.166000   3.0
BOT    6          00:01:17.400000   4.0
LEC    17         00:01:17.510000   1.0
OCO    11         00:01:17.580000   1.0
SAI    11         00:01:17.620000   1.0
RIC    11         00:01:17.622000   1.0
HAM    9          00:01:17.632000   4.0
SAI    8          00:01:17.656000   2.0
PER    11         00:01:17.669000   1.0
NOR    8          00:01:17.696000   1.0
PER    16         00:01:17.701000   2.0
LEC    14         00:01:17.708000   2.0
       11         00:01:17.717000   3.0
RIC    8          00:01:17.719000   2.0
OCO    8          00:01:17.743000   2.0
SAI    14         00:01:17.808000   3.0
NOR    5          00:01:17.821000   2.0
SAI    5          00:01:17.847000   4.0
OCO    14         00:01:17.868000   3.0
LEC    8          00:01:17.919000   4.0
ALO    8          00:01:17.966000   1.0
STR    8          00:01:17.974000   1.0
GAS    5          00:01:17.982000   1.0
BOT    2          00:01:18.005000   5.0
NOR    13         00:01:18.010000   3.0
GAS    8          00:01:18.031000   2.0
RIC    5          00:01:18.037000   3.0
OCO    5          00:01:18.041000   4.0
LEC    5          00:01:18.041000   5.0
NOR    16         00:01:18.052000   4.0
PER    8          00:01:18.069000   3.0
VET    8          00:01:18.079000   1.0
VER    2          00:01:18.090000   4.0
VET    11         00:01:18.096000   2.0
ALO    16         00:01:18.147000   2.0
GAS    2          00:01:18.190000   3.0
PER    5          00:01:18.203000   4.0
SAI    2          00:01:18.205000   5.0
STR    5          00:01:18.241000   2.0
HAM    2          00:01:18.245000   5.0
RIC    2          00:01:18.264000   4.0
ALO    5          00:01:18.281000   3.0
OCO    2          00:01:18.281000   5.0
VET    5          00:01:18.289000   3.0
GIO    11         00:01:18.356000   1.0
LEC    2          00:01:18.421000   6.0
RUS    7          00:01:18.445000   1.0
STR    11         00:01:18.467000   3.0
ALO    13         00:01:18.526000   4.0
GIO    5          00:01:18.549000   2.0
TSU    5          00:01:18.556000   1.0
ALO    2          00:01:18.560000   5.0
PER    2          00:01:18.609000   5.0
RUS    4          00:01:18.635000   2.0
TSU    2          00:01:18.648000   2.0
NOR    2          00:01:18.806000   5.0
VET    2          00:01:18.895000   4.0
RAI    5          00:01:18.917000   1.0
GIO    2          00:01:19.032000   3.0
RAI    2          00:01:19.073000   2.0
STR    2          00:01:19.077000   4.0
MSC    8          00:01:19.117000   1.0
RUS    10         00:01:19.154000   3.0
LAT    8          00:01:19.219000   1.0
MSC    5          00:01:19.405000   2.0
LAT    5          00:01:19.431000   2.0
GIO    8          00:01:19.570000   4.0
MAZ    8          00:01:19.807000   1.0
LAT    2          00:01:20.241000   3.0
MSC    2          00:01:20.309000   3.0
HAM    5          00:01:20.505000   6.0
MAZ    2          00:01:20.560000   2.0
       5          00:01:21.158000   3.0
VER    5          00:01:26.495000   5.0
MSC    6          00:01:29.913000   4.0
VER    11         00:01:30.319000   6.0
HAM    16         00:01:31.357000   7.0
MAZ    6          00:01:33.855000   4.0
ALO    14         00:01:33.967000   6.0
GIO    6          00:01:34.754000   5.0
MAZ    3          00:01:35.009000   5.0
ALO    11         00:01:35.203000   7.0
GAS    3          00:01:35.474000   4.0
RUS    2          00:01:35.607000   4.0
LEC    15         00:01:35.921000   7.0
MSC    3          00:01:36.456000   5.0
GIO    9          00:01:36.541000   6.0
STR    3          00:01:36.847000   5.0
PER    14         00:01:36.880000   6.0
GIO    12         00:01:36.888000   7.0
RIC    6          00:01:37

In [14]:
# track2[((track2['LapTime']=='00:01:16.741000') | (track2['LapTime']=='00:01:16.777000'))]

In [16]:
fig_speed=px.line(data_frame=track2[(track2['LapTime']=='00:01:16.741000')|(track2['LapTime']=='00:01:17.510000')], x="Dist_Diff_Sum", y="Speed",
           color="Driver", hover_name="Driver",
           height=900, width=1800,
           template="plotly_dark",
           title="Speed Trace For GP",
           color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
          }
          )
# fig_speed.add_layout_image(
#     dict(
#         source=Image.open("/content/images.jpg"),
#         xref="x", yref="y",
#         x=4500, y=300,
#         sizex=500, sizey=500,
#         xanchor="left", yanchor="middle"
#     )
# )
fig_speed.update_layout(
    xaxis_title="Distance (m)",
    yaxis_title="Speed (Km/Hr)",
)
fig_speed.show()

In [17]:
fig_speed_addtl=px.line(data_frame=track2[track2['LapTime']=='00:01:16.777000'], x="Dist_Diff_Sum", y="Speed",
           color="Driver", hover_name="Driver",
           height=900, width=1800,
           template="plotly_dark",
           title="Speed Trace For GP",
           color_discrete_map={
                 "HAM": "#3860C6",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
          }
          )

fig_speed.add_traces(
    list(fig_speed_addtl.select_traces())
)

fig_speed.update_layout(
    xaxis_title="Distance (m)",
    yaxis_title="Speed (Km/Hr)",
)
fig_speed.show()

In [ ]:
#Speed Trace with Corner Markings

In [ ]:
v_min = track2['Speed'].min()
v_max = track2['Speed'].max()
corner_distance=circuit_info.corners['Distance']

for k in corner_distance:
  fig_speed.add_shape(type='line',
                yref="y",
                xref="x",
                x0=k,
                y0=v_min+25,
                x1=k,
                y1=v_max+20,
                line_width=1,
                line_dash="dash",
                line_color="grey")
fig_speed.update_xaxes(range=[0, 5000])
fig_speed.update_yaxes(range=[10, 350])

for _, corner in circuit_info.corners.iterrows():
    txt = f"{corner['Number']}{corner['Letter']}"
    fig_speed.add_annotation(x=corner['Distance'], y=v_min+15, text=txt, showarrow=False, yshift=10)
fig_speed.show()


In [ ]:
fig_throttle=px.line(data_frame=track2[track2['LapTime'].isin(['00:01:16.741000','00:01:17.510000'])], x="Dist_Diff_Sum", y="Throttle",
           color="Driver", hover_name="Driver",
           height=900, width=1800,
           template="plotly_dark",
           title="Throttle Trace For GP",
           color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
          }
          )

fig_throttle.update_layout(
    xaxis_title="Distance (m)",
    yaxis_title="DRS",
)

v_min = track2['Throttle'].min()
v_max = track2['Throttle'].max()
corner_distance=circuit_info.corners['Distance']

for k in corner_distance:
  fig_throttle.add_shape(type='line',
                yref="y",
                xref="x",
                x0=k,
                y0=v_min-5,
                x1=k,
                y1=v_max+20,
                line_width=1,
                line_dash="dash",
                line_color="grey")
fig_throttle.update_xaxes(range=[0, 5000])
fig_throttle.update_yaxes(range=[-10, 110])

for _, corner in circuit_info.corners.iterrows():
    txt = f"{corner['Number']}{corner['Letter']}"
    fig_throttle.add_annotation(x=corner['Distance'], y=v_min-7, text=txt, showarrow=False, yshift=10)
fig_throttle.show()

In [ ]:
fig_brake=px.line(data_frame=track2[track2['LapTime'].isin(['00:01:16.741000','00:01:17.510000'])], x="Dist_Diff_Sum", y="Braking",
           color="Driver", hover_name="Driver",
           height=900, width=1800,
           template="plotly_dark",
           title="Brake Trace For GP",
           color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
          }
          )

fig_brake.update_layout(
    xaxis_title="Distance (m)",
    yaxis_title="Brakes",
)

v_min = track2['Braking'].min()
v_max = track2['Braking'].max()
corner_distance=circuit_info.corners['Distance']

for k in corner_distance:
  fig_brake.add_shape(type='line',
                yref="y",
                xref="x",
                x0=k,
                y0=v_min-0.25,
                x1=k,
                y1=v_max+0.2,
                line_width=1,
                line_dash="dash",
                line_color="grey")
fig_brake.update_xaxes(range=[0, 5000])
fig_brake.update_yaxes(range=[-0.5, 1.2])

for _, corner in circuit_info.corners.iterrows():
    txt = f"{corner['Number']}{corner['Letter']}"
    fig_brake.add_annotation(x=corner['Distance'], y=v_min-0.3, text=txt, showarrow=False, yshift=10)

fig_brake.show()

In [ ]:
fig_drs=px.line(data_frame=track2[track2['LapTime'].isin(['00:01:16.741000','00:01:17.510000'])], x="Dist_Diff_Sum", y="DRS",
           color="Driver", hover_name="Driver",
           height=900, width=1800,
           template="plotly_dark",
           title="DRS Trace For GP",
           color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
          }
          )

fig_drs.update_layout(
    xaxis_title="Distance (m)",
    yaxis_title="DRS",
)

v_min = track2['DRS'].min()
v_max = track2['DRS'].max()
corner_distance=circuit_info.corners['Distance']

for k in corner_distance:
  fig_drs.add_shape(type='line',
                yref="y",
                xref="x",
                x0=k,
                y0=v_min-5,
                x1=k,
                y1=v_max+2,
                line_width=1,
                line_dash="dash",
                line_color="grey")
fig_drs.update_xaxes(range=[0, 5000])
fig_drs.update_yaxes(range=[-0.5, 18])

for _, corner in circuit_info.corners.iterrows():
    txt = f"{corner['Number']}{corner['Letter']}"
    fig_drs.add_annotation(x=corner['Distance'], y=v_min-5.5, text=txt, showarrow=False, yshift=10)

fig_drs.show()

In [ ]:
#Plot as a subplot
import plotly.express as px
import plotly.subplots as sp


# For as many traces that exist per Express figure, get the
# traces from each plot and store them in an array.
# This is essentially breaking down the Express fig into it's traces
figure1_traces = []
figure2_traces = []
figure3_traces = []
figure4_traces = []

for trace in range(len(fig_speed["data"])):
    figure1_traces.append(fig_speed["data"][trace])
for trace in range(len(fig_throttle["data"])):
    figure2_traces.append(fig_throttle["data"][trace])
for trace in range(len(fig_brake["data"])):
    figure3_traces.append(fig_brake["data"][trace])
for trace in range(len(fig_drs["data"])):
    figure4_traces.append(fig_drs["data"][trace])

this_figure = sp.make_subplots(rows=4, cols=1)

for traces in figure1_traces:
    this_figure.append_trace(traces, row=1, col=1)
for traces in figure2_traces:
    this_figure.append_trace(traces, row=2, col=1)
for traces in figure3_traces:
    this_figure.append_trace(traces, row=3, col=1)
for traces in figure4_traces:
    this_figure.append_trace(traces, row=4, col=1)

this_figure.update_layout(height=1800, width=1800, title_text="Lap Traces")


this_figure.show()